In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function, absolute_import, division

In [13]:
import numpy as np
from wgomoku import (
    GomokuBoard, Move, StochasticMaxSampler, HeuristicGomokuPolicy, ThreatSearch,
    Heuristics, GomokuTools as gt, create_sample, data_from_game)

In [4]:
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U = \
    1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
BLACK=0
WHITE=1
EDGES=2

In [30]:
import pandas as pd
df = pd.read_csv("self_play_data/startwith_i10h9j9h11k10g10_style_2_ts_65.csv")
games = df['game']      
games[5]

'i10h9j9h11k10g10h10f9i12d7e8g9e9g11g8f11i11'

In [31]:
import os, tensorflow as tf
def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
heuristics = Heuristics(kappa=3.0)
p = HeuristicGomokuPolicy(style = 2, bias=.5, topn=5)
tfr_filename = os.path.join('self_play_data', "multigames-3.tfr")

with tf.python_io.TFRecordWriter(tfr_filename) as writer:
    for i, game in enumerate(games):
        stones = gt.string_to_stones(game)
        board = GomokuBoard(N=19, stones=stones, heuristics=heuristics, disp_width=10)
        smp, _, _, lbl = data_from_game(board, p, heuristics)
        print(game, smp.shape[0])
        s_and_d = list(zip(smp, lbl))

        for vec in s_and_d:
            # Create an example protocol buffer
            example = tf.train.Example(features=tf.train.Features(feature={
                'state': _floats_feature(vec[0].flatten()),
                'distr' : _floats_feature(vec[1].flatten()),
                }))
            writer.write(example.SerializeToString())
    

i10h9j9h11k10g10j7j10f9f11e12i11g11h12k9 56
i10h9j9h11k10g10j7j10i12f11e12i11g11h12h10 56
i10h9j9h11k10g10j7j10i12f11e12i11g11h12g13 56
i10h9j9h11k10g10h10f9e8i12j13g9e9g11g12f11i11 72
i10h9j9h11k10g10i8l11g6h7j10l10j8j7j11j12l8k8i11k9 96
i10h9j9h11k10g10h10f9i12d7e8g9e9g11g8f11i11 72
i10h9j9h11k10g10h7i8j7j10k8k7l7m6i6h5l9m10k9m9k12k11j5k4m8n7 144
i10h9j9h11k10g10h7i8f11k6j7j10i12j11i7k7k4k9i11i13j5i6j6j8h10k8k5 152
i10h9j9h11k10g10j7j10f9f11e12i11g11h12h10 56
i10h9j9h11k10g10h7i8f11f9e8j7k6i12j13g9e9g12g11h12h10 104
i10h9j9h11k10g10i8h7l11m12j10l10j8j7j11j12m8l9k11k8 96
i10h9j9h11k10g10j7j10i12f11e12i11g11h12h10 56
i10h9j9h11k10g10j7j10i8l11h7g6i7i9 48
i10h9j9h11k10g10j7j10l11i8k8f11e12g11i11e11d11g8g9h8h10 104
i10h9j9h11k10g10j7j10i12f11e12i11g11h12g13 56
i10h9j9h11k10g10h7i8j7j10k8k7l7m6i6l9h5g4j5g8l3k4i5k5f5g5k6i4m8n9j4j6h6g7 208
i10h9j9h11k10g10h7i8f11h10h12i12f9f10e10g8j7j13k14j8h8g11i9 120
i10h9j9h11k10g10h10f9i12d7e8g9e9g11g12f11e11 72
i10h9j9h11k10g10j7j10l11i8f11f9i12e8d7g9e

In [18]:
list(enumerate([1,2,2]))

[(0, 1), (1, 2), (2, 2)]